In [6]:
import sys
sys.path.append("../")

from src.dataloader.meta_loader import Dataloader4SER, NShotMAMLSampler, SERNShot
from src.audio.audio import AudioProcessor

from src.meta_learner.meta_learner import Meta

import matplotlib.pyplot as plt
%matplotlib inline

In [25]:
# Testing loader
N_WAY = 5
K_SHOT = 2
import torch.utils.data as data
import torch

ap = AudioProcessor(fft_size = 1024,
                    hop_length = 256,
                    win_length = 1024,
                    pad_wav=False,
                    num_mels = 80,
                    mel_fmin = 80,
                    mel_fmax = 7600,
                    sample_rate = 22050,
                    duration = None,
                    resample = True,
                    signal_norm= True,
                    ref_level_db = 20,
                    min_level_db = -100,
                    symetric_norm = True,
                    max_norm = 4)

nshot = SERNShot(df_train_path = 'df.csv', df_test_path = 'df.csv', ap = ap, batch_size = 2, n_way = N_WAY, k_shot = K_SHOT, k_query = K_SHOT//2, pad_to = 200, pad_value = -3)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:True
 | > duration:None
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:True
 | > symmetric_norm:None
 | > mel_fmin:80
 | > mel_fmax:7600
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
There are 6 samples for training and 6 for out-of-distribution language.
n_class train = 6 | n_class test = 6


..\src\dataloader\meta_loader.py:241: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self.data = np.array(self.data)
..\src\dataloader\meta_loader.py:242: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self.test_data = np.array(self.test_data)


In [150]:
import numpy as np

class ARGS:
    
    def __init__(self):
    
        self.update_lr = 0.01
        self.meta_lr = 0.01
        self.n_way = 7
        self.k_spt = 2
        self.k_qry = 1
        self.task_num = 8
        self.update_step = 5
        self.update_step_test = 10
        self.imgsz = 80
        self.imgc = 1
        self.epoch = 1
    
args = ARGS()

config = [
    ('conv2d', [32, 1, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [32, 32, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [32, 32, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [32, 32, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 1, 0]),
    ('flatten', []),
    ('linear', [args.n_way, 3200])
]


maml = Meta(args, config).cuda()

In [151]:
tmp = filter(lambda x: x.requires_grad, maml.parameters())
num = sum(map(lambda x: np.prod(x.shape), tmp))
print(maml)
print('Total trainable tensors:', num)

Meta(
  (net): Learner(
    conv2d:(ch_in:1, ch_out:32, k:3x3, stride:1, padding:0)
    relu:(True,)
    bn:(32,)
    max_pool2d:(k:2, stride:2, padding:0)
    conv2d:(ch_in:32, ch_out:32, k:3x3, stride:1, padding:0)
    relu:(True,)
    bn:(32,)
    max_pool2d:(k:2, stride:2, padding:0)
    conv2d:(ch_in:32, ch_out:32, k:3x3, stride:1, padding:0)
    relu:(True,)
    bn:(32,)
    max_pool2d:(k:2, stride:2, padding:0)
    conv2d:(ch_in:32, ch_out:32, k:3x3, stride:1, padding:0)
    relu:(True,)
    bn:(32,)
    max_pool2d:(k:2, stride:1, padding:0)
    flatten:()
    linear:(in:3200, out:7)
    
    (vars): ParameterList(
        (0): Parameter containing: [torch.cuda.FloatTensor of size 32x1x3x3 (GPU 0)]
        (1): Parameter containing: [torch.cuda.FloatTensor of size 32 (GPU 0)]
        (2): Parameter containing: [torch.cuda.FloatTensor of size 32 (GPU 0)]
        (3): Parameter containing: [torch.cuda.FloatTensor of size 32 (GPU 0)]
        (4): Parameter containing: [torch.cuda.F

In [152]:
device = 'cuda:0'

In [160]:
for step in range(args.epoch):

    x_spt, y_spt, x_qry, y_qry = nshot.next()
    x_spt, y_spt, x_qry, y_qry = torch.from_numpy(x_spt).to(device), torch.from_numpy(y_spt).type(torch.LongTensor).to(device), \
                                 torch.from_numpy(x_qry).to(device), torch.from_numpy(y_qry).type(torch.LongTensor).to(device)

    # set traning=True to update running_mean, running_variance, bn_weights, bn_bias
    accs = maml(x_spt, y_spt, x_qry, y_qry)

In [161]:
y_spt

tensor([[4, 0, 0, 4, 3, 2, 1, 1, 3, 2],
        [4, 1, 1, 0, 2, 0, 2, 3, 4, 3]], device='cuda:0')

In [139]:
accs

array([0.3, 0.3, 0.3, 0.3, 0.3, 0.3])

In [140]:
accs = []
for _ in range(10//args.task_num):
    # test
    x_spt, y_spt, x_qry, y_qry = nshot.next('test')
    x_spt, y_spt, x_qry, y_qry = torch.from_numpy(x_spt).to(device), torch.from_numpy(y_spt).to(device).type(torch.LongTensor), \
                                 torch.from_numpy(x_qry).to(device), torch.from_numpy(y_qry).to(device).type(torch.LongTensor)

    # split to single task each time
    for x_spt_one, y_spt_one, x_qry_one, y_qry_one in zip(x_spt, y_spt, x_qry, y_qry):
        test_acc = maml.finetunning(x_spt_one, y_spt_one, x_qry_one, y_qry_one)
        accs.append( test_acc )

    print("DONE!")
# [b, update_step+1]
accs = np.array(accs).mean(axis=0).astype(np.float16)
print('Test acc:', accs)


DONE!
Test acc: [0.2 0.2 0.3 0.2 0.2 0.2 0.3 0.3 0.3 0.3 0.3]


In [141]:
x_spt_one.shape

torch.Size([10, 1, 200, 80])

In [142]:
from    copy import deepcopy
net = deepcopy(maml.net)

from    torch.nn import functional as F

# 1. run the i-th task and compute loss for k=0
logits = net(x_spt_one)
loss = F.cross_entropy(logits, y_spt_one)
grad = torch.autograd.grad(loss, net.parameters())
fast_weights = list(map(lambda p: p[1] - maml.update_lr * p[0], zip(grad, net.parameters())))

In [143]:
logits_q = net(x_qry_one, fast_weights, bn_training=True)

In [144]:
logits_q.shape, x_qry_one.shape

(torch.Size([5, 7]), torch.Size([5, 1, 200, 80]))

In [145]:
pred_q = F.softmax(logits_q, dim=1).argmax(dim=1)
correct = torch.eq(pred_q, y_qry_one).sum().item()  # convert to numpy

In [146]:
test_acc

array([0.4, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2])

In [ ]:
test_acc

In [ ]:
len(accs)

In [97]:
10//args.task_num

1

In [98]:
x_qry_one.shape

torch.Size([5, 1, 200, 80])

In [99]:
x_spt_one.shape

torch.Size([10, 1, 200, 80])